<a href="https://colab.research.google.com/github/jackmiller-hash/Steam-review-scraper/blob/master/Utils/Emergency_Processor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 🛠️ Emergency Data Processor { display-mode: "form" }
#@markdown Run this if the scraper gets stuck in a "Cooldown" loop to process what you've already downloaded.

import os, json, pandas as pd
import matplotlib.pyplot as plt
from textblob import TextBlob
from wordcloud import WordCloud
from tqdm.notebook import tqdm

APP_ID = 1903340 #@param {type:"integer"}
REPORT_NAME = "Expedition33_Recovered_Report" #@param {type:"string"}

path = f'data/review_{APP_ID}.json'

if os.path.exists(path):
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    reviews = data.get('reviews', {})
    print(f"✅ Found {len(reviews)} reviews. Starting Analysis...")

    results = []
    for r_id in tqdm(reviews, desc="Processing Local Data"):
        text = reviews[r_id].get('review', '')
        if text:
            score = TextBlob(text).sentiment.polarity
            results.append({'Review': text, 'Score': score})

    df = pd.DataFrame(results)
    df.to_csv(f"{REPORT_NAME}.csv", index=False)

    # Generate Clouds
    pos_text = " ".join(df[df['Score'] > 0.2]['Review'])
    neg_text = " ".join(df[df['Score'] < -0.1]['Review'])
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    if pos_text:
        wc_pos = WordCloud(background_color="white", colormap="Greens").generate(pos_text)
        ax1.imshow(wc_pos, interpolation='bilinear'); ax1.axis("off"); ax1.set_title("FAN FAVORITES")
    if neg_text:
        wc_neg = WordCloud(background_color="white", colormap="Reds").generate(neg_text)
        ax2.imshow(wc_neg, interpolation='bilinear'); ax2.axis("off"); ax2.set_title("PAIN POINTS")
    plt.show()
    print(f"\nDONE: Report saved as {REPORT_NAME}.csv")
else:
    print("❌ No local data found. You need to run the scraper at least once first.")